In [137]:
import numpy as np
np.set_printoptions(suppress=True)
from numpy import pi
from scipy import linalg as la
import numpy as np
import os
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Some rotation utility functions

def calculate_rotation_matrix(n,sin,cos):
    '''
    Rotatation matrix around a normal vector n
    By angle theta, passed in as <sin> = sin(theta) and
    <cos> = cos(theta)
    '''
    assert np.abs(la.norm(n) - 1) < 1e-3, 'n must be a unit vector'
    
    n1 = n[0]
    n2 = n[1]
    n3 = n[2]
#     cos = np.sqrt(1-sin**2)
    R = np.zeros((3,3))
    R[0,0] = cos + n1**2 * (1-cos)
    R[0,1] = n1*n2*(1-cos) - n3*sin
    R[0,2] = n1*n3*(1-cos) + n2*sin
    R[1,0] = n1*n2*(1-cos) + n3*sin
    R[1,1] = cos + n2**2 * (1-cos)
    R[1,2] = n2*n3*(1-cos) - n1*sin
    R[2,0] = n1*n3*(1-cos) - n2*sin
    R[2,1] = n2*n3*(1-cos) + n1*sin
    R[2,2] = cos + n3**2 * (1-cos)
    
    return R

def rotate_to_z(v):
    '''
    Takes a vector v and rotates it to be aligned with the z axis
    '''
    normv = la.norm(v)
    v = v / normv
    z = np.array([0,0,1])
    # normal vector between v and z
    n = np.cross(v,z)
    # sin of angle by which to rotate
    sin = la.norm(n)
    cos = np.dot(v,z)
    n = n / sin
    R = calculate_rotation_matrix(n,sin,cos)
    vz = R @ v * normv
    return vz,n,sin

def rotate_z_to_v(z_noisy,v):
    '''
    Takes vector z_noisy and rotates it according to the same rotation
    that would align z with v
    '''
    z = np.array([0,0,1])
    n = np.cross(z,v)
    sin = la.norm(n)
    n = n / sin
    cos = np.dot(z,v)
    R = calculate_rotation_matrix(n,sin,cos)
    v_noisy = R @ z_noisy
    return v_noisy

# Setup helper functions
    
def distribute_sensors(n_sensors,lb=5,ub=15):
    '''
    distribute the sensors in a hollowed sphere
    '''
    r = np.random.uniform(lb,ub,n_sensors)
    theta = np.random.uniform(0,np.pi,n_sensors)
    phi = np.random.uniform(0,2*np.pi,n_sensors)
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    S = np.vstack((x,y,z))
    return S

def form_V(S,sig_a=0,sig_b=0,a=np.array([1,0,0]),b=np.array([0,1,0])):
    S_norm = la.norm(S,axis=0)
    V = -S / S_norm
    V = add_noise(a,b,sig_a,sig_b,V)
    return V
    
def form_G(S,V):
    '''
    Helper function for least squares triangulation
    G is the geometry matrix
    V1 is a matrix of unit pointing vectors
    '''
    _,n = S.shape
    I = np.tile(np.eye(3),(n,1))
    S_norm = la.norm(S,axis=0)
    V_temp = np.tile(V,(n,1))
    id_temp = np.kron(np.eye(n),np.ones((3,1)))
    V2 = V_temp * id_temp
    G = np.c_[I,V2]
    return G

def add_noise(a, b, sig_a, sig_b, V):
    '''
    Add noise to V along axes a, b
    Characterized by standard deviations sig_a, sig_b
    Assuming that noise is white
    For the nth sensor, assume that z points in the direction of v_n
    y is positive in the camera-up direction
    x is positive such that x cross y = z (will be in the left direction)
    a and b should be orthogonal
    '''
    assert np.dot(a,b) < 1e-3, "a and b must be orthogonal"
    _,N = V.shape
    V_noisy = np.zeros_like(V)
    z = np.array([0,0,1])
    # normal vectors for rotation
    n_a = np.cross(z,a)
    n_b = np.cross(z,b)
    for i in range(N):
        # randomly perturb z along a, b
        theta_a = np.random.normal(0,sig_a,1)
        R_a = calculate_rotation_matrix(n_a,np.sin(theta_a),np.cos(theta_a))
        theta_b = np.random.normal(0,sig_b,1)
        R_b = calculate_rotation_matrix(n_b,np.sin(theta_b),np.cos(theta_b))
        z_noisy = R_a @ R_b @ z
        
        v = V[:,i]
        assert np.abs(la.norm(v) - 1) <= 1e-3, "V must be composed of unit column vectors"
        V_noisy[:,i] = rotate_z_to_v(z_noisy, v)
    
    return V_noisy

def plot_pointing_vectors(ax, S, V):
    _,N = V.shape
    for i in range(N):
        # get endpoints
        ep1 = S[:,i]
        l = la.norm(ep1)*1.5
        ep2 = ep1 + l * V[:,i]
        ax.plot((ep1[0],ep2[0]),(ep1[1],ep2[1]),(ep1[2],ep2[2]),linewidth=1,zorder=-1)
        ax.scatter(S[0,:],S[1,:],S[2,:],c = 'm')

In [218]:
def sensor_proj(p,v):
    '''
    calculate projection matrices for the sensors
    p - sensor position
    v - pointing vector
    assumes that the pointing vector, up vector, and z axis will always be coplanar
    and that the z component of up will be positive
    '''
    tmpn = np.cross([0,0,1],v)
    tmpn = tmpn / la.norm(tmpn)
    up = np.cross(v,tmpn)
    up = up / la.norm(up)

    L = v / la.norm(v)
    tmpa = np.cross(L,up)
    tmpa /= la.norm(tmpa)
    tmpb = np.cross(tmpa,L)
    R = np.r_[tmpa,tmpb,-L].reshape((3,3))
    T = -R @ p
    P = np.c_[R,T]
    return P

def decompose(P):
    '''
    Decomposes a camera projection matrix P into K, R, and t
    '''
    M = P[:3,:3]
    Minv = la.inv(M)
    C = -Minv @ P[:3,3]
    
    # R must be orthogonal
    # K must be upper triangular
    K,R = la.rq( M )
    
    # Enforce that K has positive diagonals
    T = np.diag( np.sign( np.diag(K) ) )
    K = K @ T
    K = K / K[2,2]
    R = T @ R
    
    t = -R @ C
    
    return K,R,t

def homogenize(x):
    return np.append(x,1)

def triangulate_l1(u1,u2,P1,P2):
    K,R1,t1 = decompose(P1)
    _,R2,t2 = decompose(P2)
    C1 = -R1.T @ t1
    C2 = -R2.T @ t2
    
    # Relative rotation and translation that transform
    # a point from C1 to C2
    R = R2 @ R1.T
    t = -R2 @ R1.T @ t1 + t2
    
    # precompute for speedup
    Kinv = la.inv(K)
    
    f1 = Kinv @ u1
#     f1 /= f1[2]
    f2 = Kinv @ u2
#     f2 /= f2[2]
    
    m1 = R @ f1
    m2 = f2
    
    # unit vectors
    m1hat = m1 / la.norm(m1)
    m2hat = m2 / la.norm(m2)
    
    # normals
    n1 = np.cross(m1, t)
    n2 = np.cross(m2, t)
    n1hat = n1 / la.norm(n1)
    n2hat = n2 / la.norm(n2)
    
    # corrections of m1 and m2
    if la.norm( np.cross(m1, t) ) <= la.norm( np.cross(m2, t) ):
        m1p = m1 - np.dot(m1, n2hat) * n2hat
        m2p = m2
    else:
        m1p = m1
        m2p = m2 - np.dot(m2, n1hat) * n1hat
    
    Rf1p = m1p
    f2p = m2p
    z = np.cross(f2p, Rf1p)
    
    X1 = t + np.dot( z, np.cross(t, f2p) ) / la.norm(z)**2 * Rf1p
    X2 = np.dot( z, np.cross(t, Rf1p) ) / la.norm(z)**2 * f2p
    return R2.T @ (X2-t2)

def triangulate_l2(u1,u2,P1,P2):
    K,R1,t1 = decompose(P1)
    _,R2,t2 = decompose(P2)

    # Relative rotation and translation that transform
    # a point from C1 to C2
    R = R2 @ R1.T
    t = -R2 @ R1.T @ t1 + t2
    
    # precompute for speedup
    Kinv = la.inv(K)
    
    m1 = R @ Kinv @ u1
    m2 = Kinv @ u2
    
    # unit vectors
    m1hat = m1 / la.norm(m1)
    m2hat = m2 / la.norm(m2)
    that = t / la.norm(t)
    
    # SVD matrix
    MT = np.vstack( (m1hat,m2hat) )
    P = np.eye(3) - np.outer(that,that)
    A = MT @ P
    _,S,Vh = la.svd(A)
    
    # Take the right singular vector corresponding to the smallest SV
    if S[1] > 1e-4:
        nhat = Vh[1]
    else:
        nhat = Vh[0]
    
    # corrections of m1 and m2
    m1p = m1 - np.dot(m1, nhat) * nhat
    m2p = m2 - np.dot(m2, nhat) * nhat
    
    Rf1p = m1p
    f2p = m2p
    z = np.cross(f2p, Rf1p)
    
    X1 = t + np.dot( z, np.cross(t, f2p) ) / la.norm(z)**2 * Rf1p
    X2 = np.dot( z, np.cross(t, Rf1p) ) / la.norm(z)**2 * f2p
    return R2.T @ (X2-t2)

In [215]:
def poseFromPointing(v,p):
    '''
    calculate projection matrices for the sensors
    v - sensor focal center
    p - pointing vector
    assumes that the pointing vector, up vector, and z axis will always be coplanar
    and that the z component of up will be positive
    '''
    tmpn = np.cross([0,0,1],p)
    tmpn = tmpn / la.norm(tmpn)
    up = np.cross(p,tmpn)
    up = up / la.norm(up)

    L = p / la.norm(p)
    tmpa = np.cross(L,up)
    tmpa /= la.norm(tmpa)
    tmpb = np.cross(tmpa,L)
    R = np.r_[tmpa,tmpb,-L].reshape((3,3))
    t = -R @ v
    return R,t

def triangulateFromPointingL1(v1,p1,v2,p2):
    R1,t1 = poseFromPointing(v1,p1)
    R2,t2 = poseFromPointing(v2,p2)
    C1 = -R1.T @ t1
    C2 = -R2.T @ t2
    
    # Relative rotation and translation that transform
    # a point from C1 to C2
    R = R2 @ R1.T
    t = -R2 @ R1.T @ t1 + t2
    
    f1 = np.array([0,0,1])
    f2 = np.array([0,0,1])
    
    m1 = R @ f1
    m2 = f2
    
    # unit vectors
    m1hat = m1 / la.norm(m1)
    m2hat = m2 / la.norm(m2)
    
    # normals
    n1 = np.cross(m1, t)
    n2 = np.cross(m2, t)
    n1hat = n1 / la.norm(n1)
    n2hat = n2 / la.norm(n2)
    
    # corrections of m1 and m2
    if la.norm( np.cross(m1, t) ) <= la.norm( np.cross(m2, t) ):
        m1p = m1 - np.dot(m1, n2hat) * n2hat
        m2p = m2
    else:
        m1p = m1
        m2p = m2 - np.dot(m2, n1hat) * n1hat
    
    Rf1p = m1p
    f2p = m2p
    z = np.cross(f2p, Rf1p)
    
    X1 = t + np.dot( z, np.cross(t, f2p) ) / la.norm(z)**2 * Rf1p
    X2 = np.dot( z, np.cross(t, Rf1p) ) / la.norm(z)**2 * f2p
    return R2.T @ (X2-t2)

def triangulateFromPointingL2(v1,p1,v2,p2):
    R1,t1 = poseFromPointing(v1,p1)
    R2,t2 = poseFromPointing(v2,p2)
    C1 = -R1.T @ t1
    C2 = -R2.T @ t2
    
    # Relative rotation and translation that transform
    # a point from C1 to C2
    R = R2 @ R1.T
    t = -R2 @ R1.T @ t1 + t2
    
    f1 = np.array([0,0,1])
    f2 = np.array([0,0,1])
    
    m1 = R @ f1
    m2 = f2
    
    # unit vectors
    m1hat = m1 / la.norm(m1)
    m2hat = m2 / la.norm(m2)
    that = t / la.norm(t)
    
    # SVD matrix
    MT = np.vstack( (m1hat,m2hat) )
    P = np.eye(3) - np.outer(that,that)
    A = MT @ P
    _,_,Vh = la.svd(A)
    nhat = Vh[1]
    
    # corrections of m1 and m2
    m1p = m1 - np.dot(m1, nhat) * nhat
    m2p = m2 - np.dot(m2, nhat) * nhat
    
    Rf1p = m1p
    f2p = m2p
    z = np.cross(f2p, Rf1p)
    
    X1 = t + np.dot( z, np.cross(t, f2p) ) / la.norm(z)**2 * Rf1p
    X2 = np.dot( z, np.cross(t, Rf1p) ) / la.norm(z)**2 * f2p
    return R2.T @ (X2-t2)

In [221]:
X = np.array([1,3,5]) # ground truth target

v1 = np.array([0,100,3])
p1 = np.array([0,-1,0])
v2 = np.array([50,50,2])
p2 = np.array([-1,-2,0])

P1 = sensor_proj(v1,p1)
P2 = sensor_proj(v2,p2)
u1 = P1 @ homogenize(X)
u1 /= u1[2]
u1 += np.array([0.0001,0.0001,0])
u2 = P2 @ homogenize(X)
u2 /= u2[2]

print("L1 solution: ", triangulate_l1(u1,u2,P1,P2))
print("L2 solution: ", triangulate_l2(u1,u2,P1,P2))

L1 solution:  [1.09619189 3.09226569 4.9941107 ]
L2 solution:  [52.23884865 47.76089805  1.95113702]


In [223]:
X = np.array([1,3,5]) # ground truth target

v1 = np.array([0,100,3])
p1 = X - v1
v2 = np.array([50,50,2])
p2 = X - v2 + [.001,0,0]

print(triangulateFromPointingL1(v1,p1,v2,p2))
print(triangulateFromPointingL2(v1,p1,v2,p2))

[1.00001147 2.99905179 5.00006052]
[1.00001091 2.99905151 5.00004704]


In [116]:
# Test 1
v1 = np.array([6800000,-2e4,0])
p1 = np.array([-0.9984, 0.0499, -0.025])
v2 = np.array([6800000,2e4,0])
p2= np.array([-0.9984, -0.0499, 0.025])
print(triangulateFromPointingL1(v1,p1,v2,p2))
print(triangulateFromPointingL2(v1,p1,v2,p2))


[ 6.40009058e+06 -1.25400962e+01 -1.00137575e+04]
[ 6.39983968e+06 -4.19165171e-11  9.57105662e-11]
